In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2
from data_loader import load_datasets, create_data_loaders
import torch
import os
from cnn_model import Net as init_cnn
import numpy as np
from init_nets import init_resnet, init_efficientnet
import torch.nn.functional as F
from serialization import save, load

SEED = 42

In [2]:
# model paths
cnn_model_path = "output/models/cnn/cnn"
resnet_model_path = "output/models/resnet/resnet"
efficientnet_model_path = "output/models/efficientnet/efficientnet"
dropout = 0.4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dir_ensemble = "output/history/ensemble/"
path_ensemble_hard = "output/history/ensemble/history_hard.pkl"
path_ensemble_soft = "output/history/ensemble/history_soft.pkl"

if not os.path.exists(dir_ensemble):
    os.makedirs(dir_ensemble)

In [ ]:
batch_size = 128
size_cnn = (32, 32)
size = (224, 224)
train_cnn, val_cnn, test_cnn = load_datasets(size_cnn)
train, val, test = load_datasets(size)

_, _, test_dataloader_cnn = create_data_loaders(train_cnn, val_cnn, test_cnn, batch_size)
_, _, test_dataloader = create_data_loaders(train, val, test, batch_size)

labels = torch.tensor(test.targets).to(device)

In [ ]:
def get_proba(model, dataloader, device):
    all_outputs = []
    model.eval()
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            output = F.softmax(output, dim=1)
            all_outputs.append(output)
    return torch.cat(all_outputs, dim=0)

def get_prediction(proba):
    _, predicted = torch.max(proba, 1)
    return predicted

def compute_accuracy(preds, labels):
    accuracy = preds.eq(labels).sum().item() / labels.size(0) * 100
    return accuracy


In [ ]:
# ensemble models
hard_accuracy_list = []
soft_accuracy_list = []

for i in range(1, 6):
    # set paths
    suffix = f"_{i}.pth"
    cnn_model_path_i = cnn_model_path + suffix
    resnet_model_path_i = resnet_model_path + suffix
    efficientnet_model_path_i = efficientnet_model_path + suffix
    
    # initialize nets
    cnn = init_cnn(dropout=dropout).to(device)
    resnet = init_resnet().to(device)
    efficientnet = init_efficientnet().to(device)
    
    # load weights
    cnn.load_state_dict(torch.load(cnn_model_path_i, weights_only=True))
    resnet.load_state_dict(torch.load(resnet_model_path_i, weights_only=True))
    efficientnet.load_state_dict(torch.load(efficientnet_model_path_i, weights_only=True))
    
    # get probabilities
    cnn_prob = get_proba(cnn, test_dataloader_cnn, device) # different data loader for cnn
    resnet_prob = get_proba(resnet, test_dataloader, device)
    efficientnet_prob = get_proba(efficientnet, test_dataloader, device)
    
    # hard voting
    cnn_pred = get_prediction(cnn_prob)
    resnet_pred = get_prediction(resnet_prob)
    efficientnet_pred = get_prediction(efficientnet_prob)
    
    models_pred = torch.stack([cnn_pred, resnet_pred, efficientnet_pred], dim=0)
    hard_pred, _ = torch.mode(models_pred, dim=0)
    hard_accuracy = compute_accuracy(hard_pred, labels)
    hard_accuracy_list.append(hard_accuracy)
    
    # soft voting
    ensemble_prob = (cnn_prob + resnet_prob + efficientnet_prob) / 3
    soft_pred = get_prediction(ensemble_prob)
    soft_accuracy = compute_accuracy(soft_pred, labels)
    soft_accuracy_list.append(soft_accuracy)

# saving results
save(hard_accuracy_list, path_ensemble_hard)
save(soft_accuracy_list, path_ensemble_soft)

In [ ]:
print(hard_accuracy_list)
print(soft_accuracy_list)

In [3]:
# total results
cnn_history_path = "output/history/cnn/cnn"
resnet_history_path = "output/history/resnet/resnet"
efficientnet_history_path = "output/history/efficientnet/efficientnet"

cnn_accuracy_list = []
resnet_accuracy_list = []
efficientnet_accuracy_list = []
hard_accuracy_list = load(path_ensemble_hard)
soft_accuracy_list = load(path_ensemble_soft)

for i in range(1, 6):
    # set paths
    suffix = f"_{i}.pkl"
    cnn_history_path_i = cnn_history_path + suffix
    resnet_history_path_i = resnet_history_path + suffix
    efficientnet_history_path_i = efficientnet_history_path + suffix
    
    cnn_test_acc = load(cnn_history_path_i)["accuracy_test"]
    cnn_accuracy_list.append(cnn_test_acc)
    
    resnet_test_acc = load(resnet_history_path_i)["accuracy_test"]
    resnet_accuracy_list.append(resnet_test_acc)
    
    efficientnet_test_acc = load(efficientnet_history_path_i)["accuracy_test"]
    efficientnet_accuracy_list.append(efficientnet_test_acc)

In [4]:
import pandas as pd

models = ["CNN", "ResNet", "EfficientNet", "soft ensemble", "hard ensemble"]
data = np.array([cnn_accuracy_list, resnet_accuracy_list, efficientnet_accuracy_list, soft_accuracy_list, hard_accuracy_list]).T
print(pd.DataFrame(data, columns=models).describe())

             CNN     ResNet  EfficientNet  soft ensemble  hard ensemble
count   5.000000   5.000000      5.000000       5.000000       5.000000
mean   61.050222  79.757111     88.954889      88.159333      84.740889
std     2.962408   2.816411      0.166353       0.332626       1.023921
min    58.560000  75.270000     88.687778      87.675556      83.376667
25%    58.668889  78.880000     88.916667      87.990000      84.032222
50%    59.546667  80.738889     88.998889      88.240000      85.110000
75%    63.667778  81.588889     89.055556      88.376667      85.244444
max    64.807778  82.307778     89.115556      88.514444      85.941111
